In [1]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup
html = urlopen('https://en.wikipedia.org/wiki/java')
bs=BeautifulSoup(html,'html.parser')
table = bs.find_all('table', {'class', 'wikitable'})[0]
rows= table.find_all('tr')
csvFile = open('c:/data/csv/editors.csv', 'w', newline='', encoding='utf-8')
writer = csv.writer(csvFile)
try:
    for row in rows:
        csvRow=[]
        for cell in row.find_all(['td', 'th']):
            csvRow.append(cell.text.strip())
        writer.writerow(csvRow)
finally:
    print('저장되었습니다.')
    csvFile.close()

저장되었습니다.


In [8]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import re
import random
import pymysql

# DB 접속
conn = pymysql.connect(host='localhost', port=3306, user='web', password='1234', db='myweb', charset='utf8')
cursor = conn.cursor()
# DB 생성
cursor.execute("use myweb")
cursor.execute("delete from pages")

# insert 함수
def store(title, content):
    # 따옴표 처리
    title = title.replace("'", "''") # 작은 따옴표 1개를 2개로
    content = content.replace("'", "''")
    sql = "insert into pages (title, content) values ('%s', '%s')" % (title, content)
    cursor.execute(sql)
    conn.commit()


# 위키피디아의 url 수집
def getLinks(url):
    html = urlopen("http://ko.wikipedia.org" + url)

    obj = bs(html, "html.parser")
    # h1 태그의 내용
    title = obj.find("h1").text
    # id가 mw-content-text인 div 태그 내부의 p 태그의 텍스트
    content = obj.find("div", {"id":"mw-content-text"}).find("p").text
    # 테이블에 저장
    store(title, content)
    # 링크를 리턴함(정규표현식 사용, ^시작, $끝, * 0회 이상 반복)
    return obj.find("div", {"id": "bodyContent"}).find_all("a", href=re.compile("^(/wiki/)((?!:).)*$"))

# 함수 호출(주소창에 한글을 입력하면 코드에서는 url encoding 처리됨)
links = getLinks("/wiki/%EC%9E%90%EB%B0%94")
try:
    cnt = 0
    while True:
        cnt += 1
        if cnt >= 50: #10건만 처리
            break
        # url 리스트 중에서 랜덤으로 url 선택
        newArticle = links[random.randint(0, len(links)-1)].attrs["href"]
        print(newArticle)
        #해당 문서에 포함된 새로운 링크를 가져옴
        links = getLinks(newArticle)
finally:
    cursor.close()
    conn.close()
    print("완료되었습니다.")

/wiki/%EA%B0%9D%EC%B2%B4_%EC%A7%80%ED%96%A5_%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%98%EB%B0%8D
/wiki/%EC%BB%B4%ED%93%A8%ED%84%B0_%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%98%EB%B0%8D
/wiki/%EA%B0%9D%EC%B2%B4_%EC%A7%80%ED%96%A5_%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%98%EB%B0%8D
/wiki/%EC%98%A4%EB%B8%8C%EC%A0%9D%ED%8B%B0%EB%B8%8C-C
/wiki/%ED%81%B4%EB%9E%AD
/wiki/%EC%98%A4%EB%B8%8C%EC%A0%9D%ED%8B%B0%EB%B8%8C-C
/wiki/%EC%8A%A4%EB%AA%B0%ED%86%A0%ED%81%AC
/wiki/%EB%A7%A4%ED%82%A8%ED%86%A0%EC%8B%9C
/wiki/%EC%8B%9C%EC%8A%A4%ED%85%9C_7
/wiki/%EA%B0%80%EC%83%81_%EB%A9%94%EB%AA%A8%EB%A6%AC
/wiki/%EC%93%B0%EB%A0%88%EA%B8%B0_%EC%88%98%EC%A7%91_(%EC%BB%B4%ED%93%A8%ED%84%B0_%EA%B3%BC%ED%95%99)
/wiki/%EC%9A%B4%EC%98%81%EC%B2%B4%EC%A0%9C
/wiki/%EC%88%98%ED%8D%BC%EB%B0%94%EC%9D%B4%EC%A0%80_%EB%AA%A8%EB%93%9C
/wiki/%EC%9A%B4%EC%98%81%EC%B2%B4%EC%A0%9C_%EC%8B%9C%EC%9E%A5_%EC%A0%90%EC%9C%A0%EC%9C%A8
/wiki/FreeBSD
/wiki/%EC%9B%A8%EC%9D%B4%EB%B0%B1_%EB%A8%B8%EC%8B%A0
/wiki/%EA%B9%A8%EC%A7%84_%EB%A7%81%ED%81%AC
/wiki/%ED%8D%BC

In [9]:
len(links)

9